# 효율적인 정리를 위한 class화

In [108]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm

class Model:
    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
        self._build_net()
    
    def _build_net(self):
        with tf.variable_scope(self.name):
            self.X = tf.placeholder(tf.float32, [None, 784])
            X_img = tf.reshape(self.X, [-1, 28, 28, 1])
            self.Y = tf.placeholder(tf.float32, [None, 10])
            self.W1 =tf.Variable(tf.random_normal([3,3,1,32], stddev=0.01))
    
    def predict(self, x_test, keep_prop=1.0):
        return self.sess.run(self.logits, feed_dict = {self.X: x_test, self.keep_prob: keep_prop})
    
    def get_accuracy(self, x_test, y_test, keep_prop=1.0):
        return self.sess.run(self.accuracy, feed_dict={self.X: x_test, self.Y: y_test, self.keep_prob: keep_prop})
    
    def train(self, x_data, y_data, learning_rate = 0.1, keep_prop=0.7):
        self.hypothesis = tf.nn.softmax(tf.matmul(self.X, self.W1)+self.b)
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.hypothesis, labels=self.Y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
        return self.sess.run([self.cost, self,optimizer], feed_dict={self.X: x_data, self.Y: y_data, self.keep_prob: keep_prop})

In [109]:
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
nb_classes = 10

In [110]:
input_shape = X_train.shape[1] * X_train.shape[2]
X_train = X_train.reshape(-1, input_shape) 
y_train = to_categorical(y_train, nb_classes)
X_train = tf.image.convert_image_dtype(X_train, tf.float32)
y_train = tf.image.convert_image_dtype(y_train, tf.float32)

In [111]:
sess = tf.InteractiveSession()
m1 = Model(sess, "m1")

In [112]:
sess.run(tf.global_variables_initializer())
print("Learning Strarted!")

Learning Strarted!


In [113]:
training_epochs = 10
batch_size = 100

In [114]:
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(60000 / batch_size)
    
    for i in range(total_batch):
        batch_xs, batch_ys = X_train[(i*batch_size):batch_size+(i*batch_size)].eval(), y_train[(i*batch_size):batch_size+(i*batch_size)].eval()
        c, _ = m1.train(batch_xs, batch_ys)
        avg_cost += c / total_batch

ValueError: Dimensions must be equal, but are 784 and 1 for '{{node MatMul_2}} = BatchMatMulV2[T=DT_FLOAT, adj_x=false, adj_y=false](m1_16/Placeholder, m1_16/Variable/read)' with input shapes: [?,784], [3,3,1,32].